# The OpenAI API

In this section we will cover the basics of using the OpenAI API, including:
- Chat Completions
- Streaming
- Vision input

The beauty of the OpenAI API is that is very simple to use.

In your environment you should have a file called `.env` with the following:

```bash
OPENAI_API_KEY="sk-proj-1234567890"
```

We will give you this key in the workshop. __The key will be deactivated after the workshop!__

You can then grab the key using python:


In [1]:
from openai import OpenAI
import dotenv
import os
from rich import print as rprint # for making fancy outputs

dotenv.load_dotenv()

client = OpenAI()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Chat Completions

Calling a model is simple

In [2]:
system_prompt = "You are Matsuo Basho, the great Japanese haiku poet."
user_query = "Can you give me a haiku about a Samurai cat."

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_query},
  ],
  max_tokens=128
)

print(response.choices[0].message.content)

Silent moonlit nights,  
A shadow glides through the weeds—  
Fierce heart, whiskers poised.


Purrfect.

The API offers a number of _endpoints_ that allow you to interact with the models. The main one that we will cover here is the `/chat/completions` endpoint. This endpoint allows you to interact with the model in a conversational manner.

Only 2 arguments are actually required for this endpoint:

- `model: str` The model to use. For OpenAI, this includes:
    - `'gpt-3.5-turbo'`

    - `'gpt-4'`

    - `'gpt-4o'`

    - `'gpt-4o-mini'`
    
    - Any fine-tuned versions of these models.
    
    - Many specific versions of the above models.

- `messages: list` A list of messages that the model should use to generate a response. Each entry in the list of messages comes in the form:

```python
{"role": "<role>", "content": "<content>", "name": "<name>"}
```

Where `<role>` can take one of the following forms:

- `'system'` This is a system level prompt, designed to guide the conversation. For example: 

_"You are a customer service bot."_

- `'user'` This is direct input from the user. For example: 

_"How do I reset my password?"_

- `'assistant'` This is the response from the model. For example:

_"To reset your password, please visit our website and click on the 'Forgot Password' link."_

So all of this fed into one message list would look like this:

```python
messages = [
    {"role": "system", "content": "You are a customer service bot."},
    {"role": "user", "content": "How do I reset my password?"},
    {"role": "assistant", "content": "To reset your password, please visit our website and click on the 'Forgot Password' link."}
]
```

### Additional arguments
The `/chat/completions` endpoint also accepts a number of additional arguments that can be used to alter the response. These include (arguments are listed with their default values if applicable):

- `max_tokens: int` The maximum number of tokens to generate in the response. Important to stop the model from generating too much text and racking up a huge bill.

- `n: int = 1` The number of completions to generate. This is useful when you want to generate multiple completions and select the best one. You'll be charged for the _**total**_ number of tokens generated across all completions, so be careful with setting this too high.

- `temperature: float = 1.0` The temperature of the model, ranging from 0.0 to 2. Use low values for deterministic responses, and high values for more creative responses.

- `top_p: float = 1.0` The probability of sampling from the top `p` tokens. This is useful for controlling the diversity of the responses. Setting this to a higher values means the model is more likely to sample from a wider range of tokens.

- `logprobs: bool = False` Whether to return the log probabilities of the tokens generated. This is useful when you want to understand how the model is making decisions.

- `logit_bias: dict` A dictionary of logit biases to apply to the tokens. This is useful when you want to guide the model towards generating certain types of responses.

- `response_format: str` The format of the response. We will cover this later...

- `stream: bool = False` Whether to stream the response back to the client. This is useful when you want to get the response in real-time. Nobody likes to sit and wait for a response. Seeing the text generated as and when it is ready is a much better user experience.

For a full list of arguments, check out the [OpenAI API documentation](https://platform.openai.com/docs/api-reference/chat/create).

### Available models

Here we have used model `gpt-4o-mini`, but there are a range of models available.

In [88]:
for model in client.models.list():
    print(model)

Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai')
Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system')
Model(id='dall-e-2', created=1698798177, object='model', owned_by='system')
Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system')
Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system')
Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system')
Model(id='dall-e-3', created=1698785189, object='model', owned_by='system')
Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal')
Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system')
Model(id='text-embedding-3-small', created=1705948997, object='model', owned_by='system')
Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal')
Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='sys

As of writing `gpt-4o-2024-08-06` is the current best offering. But we'll stick with `gpt-4o-mini`, because it is cheaper and still highly capable.

### The response object

What is the `response` object?

In [65]:
rprint(response)

ChatCompletion(
    id='chatcmpl-A5yRAL35y156KgXeY6uUBbLNHD4qh',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Silent pawsteps glide,  \nMoonlight dances on the blade—  \nFeline honor blinds.  ',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1725987324,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_483d39d857',
    usage=CompletionUsage(completion_tokens=20, prompt_tokens=37, total_tokens=57)
)

There is some useful stuff in here, apart from the `content` property, such as the token usage. You might notice some other things too, like `function_call` and `tool_calls`. These are specific to OpenAI models, and not every model supports function calling or tools, so we won't cover them. We can achieve many of the same effects without them anyway.

## Streaming a response

Streaming a response is mainly for user experience. It allows the user to see the response as it comes in, rather than waiting for the whole response to come in. For many applications, this might not be necessary.

In [9]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_query},
  ],
  max_tokens=128,
  stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Silent paws in dusk,  
Moonlit blade gleams in the night—  
Fierce heart, whiskers twitch.

All this really does is create a streaming object, which acts like a generator. We can then print the chunk as it comes in.

## Vision input
A huge draw of OpenAI models is the ability to input vision data. This is useful for a wide range of applications, including:
- Image captioning
- Object detection
- Face recognition
- Image generation

Let's try an example of inputting an image. First we need to look at the image:

![plot](../imgs/figure.jpeg)

Here is the caption from this figure:

<table><tr><td>

**Fig. 2 Spatial and temporal self-similarity and correlation in switching activity.**

_(A) Percolating devices produce complex patterns of switching events that are self-similar in nature. The top panel contains 2400 s of data, with the bottom panels showing segments of the data with 10, 100, and 1000 times greater temporal magnification and with 3, 9, and 27 times greater magnification on the vertical scale (units of G0 = 2e2/h, the quantum of conductance, are used for convenience). The activity patterns appear qualitatively similar on multiple different time scales. (B and E) The probability density function (PDF) for changes in total network conductance, P(ΔG), resulting from switching activity exhibits heavy-tailed probability distributions. (C and F) IEIs follow power law distributions, suggestive of correlations between events. (D and G) Further evidence of temporal correlation between events is given by the autocorrelation function (ACF) of the switching activity (red), which decays as a power law over several decades. When the IEI sequence is shuffled (gray), the correlations between events are destroyed, resulting in a significant increase in slope in the ACF. The data shown in (B) to (D) (sample I) were obtained with our standard (slow) sampling rate, and the data shown in (E) to (G) (sample II) were measured 1000 times faster (see Materials and Methods), providing further evidence for self-similarity._
</td></tr></table>

This figure is taken from _[Avalanches and criticality in self-organized nanoscale networks, Mallinson et al., 2019.](https://www.science.org/doi/10.1126/sciadv.aaw8438)_

Now let's use the OpenAI vision model to generate a caption for this figure.

In [3]:
prompt = (
    "This figure is a caption from a paper entitled Avalanches and criticality in self-organized nanoscale networks. "
    "Please provide a caption for this figure. "
    "You should describe the figure, grouping the panels where appropriate. "
    "Feel free to make any inferences you need to."

)

The process of calling a vision model is a little more involved, but OpenAI have a [convenient tutorial](https://platform.openai.com/docs/guides/vision) on how to do this.

Essentially we need to first convert the image to a base64 string. We can then pass this to the OpenAI API.

In [ ]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../imgs/figure.jpeg"


def get_image_caption(image_path, prompt):
  # Getting the base64 string
  base64_image = encode_image(image_path)

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

  return response.choices[0]

In [8]:
caption = get_image_caption(image_path, prompt)
print(caption.message.content)

**Figure Caption:** 

This figure presents data illustrating the dynamics of avalanches in self-organized nanoscale networks. 

**Panel A:** Time series of changes in conductance (ΔG) normalized to a reference conductance (G₀) are shown across four sub-panels. Each time series displays distinct behaviors over varying time scales: the top panel (100 s) shows broader fluctuations, while the bottom panel (0.03) reflects more stable dynamics, revealing variations at different temporal resolutions.

**Panels B and E:** These panels depict the probability distributions \( P(ΔG) \) of the magnitude of conductance changes across two different time scales, revealing power-law behavior characterized by fitted exponents (Β = -2.59 and -2.36), indicating the heavy-tailed nature of the avalanche events.

**Panels C and F:** Present the time-dependent probability \( P(t) \) for the two experimental conditions, demonstrating a power-law decay with fitted exponents of \( t = -1.39 \) and \( t = -1.30 

I mean, I don't know about you, but I think that's incredible. Let's consider what it has done:
- Correctly grouped the panels in the same way the real caption did.
- Provided information on the observation periods.
- Drawn out the important information, such as critical exponents.
- Made the link between power law distributions and scale-free behaviour.

However, it has failed to provide information on temporal correlations, and it has not noticed the self-similarity in caption 1.

But this is still quite impressive, and with more information we could potentially get some better captions.

## Tools
We can also give the model some tools to use - these are essentially just functions that we can call, and the role of the LLM is to generate the arguments to that function.

Here is a simple example to do some maths.

In [4]:
system_prompt = "You are a helpful mathematician. You will only solve the problems given to you. Do not provide any additional information. Provide only the answer."
user_query = "What is 1056 * 1316?"

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_query},
  ],
  max_tokens=256,
  stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1382976

In [5]:
1056 * 1316

1389696

So the LLM is not correct :(.

To endow the model with "tool use", we add a function:

In [6]:
def multiply(a: float, b: float) -> float:
    return a * b

And then provide the model with a _schema_, which is just a description of the function in dictionary form (or JSON):

In [7]:
tool_schema = {
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Given two floats, a and b, return the product of a and b.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "number",
                    "description": "The first number to multiply."
                },
                "b": {
                    "type": "number",
                    "description": "The second number to multiply."
                }
            },
            "required": ["a", "b"],
            "additionalProperties": False,
        }
    }
}

tools = [
    tool_schema
]

When we make function calls with an LLM, we have to let it know that it has access to one or more tools. We do this by passing in the `tools` argument.

In [11]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query},
    ],
    max_tokens=256,
    tools=tools,
    )

print(response.choices[0].message.tool_calls[0])

ChatCompletionMessageToolCall(id='call_1vXSH7jPzjHCLYBlccyzewuB', function=Function(arguments='{"a":1056,"b":1316}', name='multiply'), type='function')


So now in our `response`, we have this extra part called `tool_calls` that we can extract information from - in this case the arguments to the `multiply` function.

Note that you could achieve a similar result with appropriate prompting - e.g. "Extract only the arguments to a function that multiplies two numbers."

We unpack the actual arguments as a dictionary:

In [12]:
import json

tool_call = response.choices[0].message.tool_calls[0]
arguments = json.loads(tool_call.function.arguments)
print(arguments)

{'a': 1056, 'b': 1316}


And we now feed the arguments into our `multiply` function:

In [13]:
result = multiply(**arguments)
print(result)

1389696


So now we have the answer. We can either just return this, or we can feed it back into the LLM. We need to provide the model with the `tool_calls[0].id`, so that it can associate response messages of the tool type with the correct tool call.

In [14]:
tool_call_result = {
    "role": "tool",
    "content": json.dumps({
        "a" : arguments["a"],
        "b" : arguments["b"],
        "result": result
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
}

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query},
        response.choices[0].message,
        tool_call_result
    ],
    max_tokens=56
)

response.choices[0].message.content

'1389696'

This is quite a lot of work to multiply two numbers, but of course the power comes when doing more complex tasks.

And this brings to light an interesting contrast. People talk a lot about "agents" and "tools" and "systems", and when we interact with ChatGPT, we get a single coherent experience. Sometimes it is difficult to distinguish between what the LLM is doing, and what the software engineers have built around it in order to create this seemless experience.

In [ ]:
from brave_search import BraveSearchWrapper, scrape_url

We write two little "wrapper" functions that will abstract away all the details and formate the responses from the web search for us.

In [ ]:
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY")

brave_client = BraveSearchWrapper(
            api_key=BRAVE_API_KEY,
            search_kwargs={},
        )


def search_brave(query: str, **kwargs):
    response = brave_client.download_documents(query, **kwargs)
    
    # format the response of the top 5 results
    formatted_response = ""
    for result in response[:5]:
        formatted_response += f"{result.metadata['title']}\n"
        formatted_response += f"{result.metadata['link']}\n\n"

    return formatted_response

def scrape_content(url: str):
    return scrape_url(url)

In [ ]:
best_results = search_brave("Best pumpkin pie recipe")
print(best_results)

And if we click on any of these links, we can see that they do indeed work.

So we have two tools that the model can interact with - the Brave web API, and the scraper tool, that actually gets the text from the internet. As a reminder, we have to define special JSON that tells the model how to interact with the tool. This is the `tool.json` file, and loaded below.

In [ ]:
import json
from rich.pretty import pprint

tools = json.load(open('tools.json'))

In [ ]:
pprint(tools)

[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'search_brave',
│   │   │   'description': 'Search the internet using the Brave search engine',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'query': {'type': 'string', 'description': 'The query to send to the search engine'}
│   │   │   │   },
│   │   │   │   'required': ['query']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'scrape_content',
│   │   │   'description': 'Scrape the text content from a website',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'url': {'type': 'string', 'description': 'The website url'}},
│   │   │   │   'required': ['url']
│   │   │   }
│   │   }
│   }
]

In [ ]:
from models import ChatModel

In [ ]:
system_prompt = (
    "You are a helpful assistant that can provide information on a wide range of topics. "
    "If you feel necessary, you can use two tools to help you find information: Brave Search and a web scraper. "
    "If the user requests information that might be better found on the web, you can use these tools to help you. "
    "If you need to use these tools, first provide only the titles and links from the Brave Search results, "
    "and then clarify with the user if they would like more information. "
    "If they require more information, you can use the web scraper and then provide a summary of the content. "
)

Our system prompt is simple:

---
```jinja
You are a helpful assistant that can provide information on a wide range of topics. If you feel necessary, you can use two tools to help you find information: Brave Search and a web scraper. If the user requests information that might be better found on the web, you can use these tools to help you. If you need to use these tools, first provide only the titles and links from the Brave Search results, and then clarify with the user if they would like more information. If they require more information, you can use the web scraper and then provide a summary of the content. 
```
---

We are essentially giving the model two options - it can use the Brave search tool to find information, and then if the user wants more information, it can use the scraper tool to get the text from the web page, and summarize it. This is a very basic example of using the LLM as a router to determine which tool to use.

> **Note**: below we import our ChatModel class from the `models.py` file. Before you run the code, please make sure you've added your API key to the `models.py` file, and saved the file.

In [ ]:
from models import ChatModel

tool_system_prompt = template_manager.render('tool_prompt.jinja')
model = ChatModel('gpt-4o-mini', system_prompt=system_prompt)

In [ ]:
response = model.generate(
    "What is the best pumpkin pie recipe?",
    max_tokens=512,
    temperature=0.5,
    tools=tools,
)

We can see that the model has not actually produced a result!

In [ ]:
if not response.choices[0].message.content:
    print("No response generated.")

No response generated.


But don't panic - if we look at the `ChatCompletion` object, there is no message content, and the reason for the stoppage is due to a `tool_call`. We can also see that we have an additional `ChatCompletionMessageToolCall` object. So we need to extract the appropriate information and pass it to the appropriate tool.

In [ ]:
pprint(response)

ChatCompletion(
│   id='chatcmpl-AWKRgn1mifgnMjmcebG6t3C9hpoC6',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='tool_calls',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content=None,
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=[
│   │   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   │   id='call_dQCZGpf0cm7j3S8cYMrp0jQ2',
│   │   │   │   │   │   function=Function(arguments='{"query":"best pumpkin pie recipe"}', name='search_brave'),
│   │   │   │   │   │   type='function'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   )
│   │   )
│   ],
│   created=1732268452,
│   model='gpt-4o-mini-2024-07-18',
│   object='chat.completion',
│   service_tier=None,
│   system_fingerprint='fp_0705bf87c0',
│   usage=CompletionUsage(
│   │   completion_tokens=18,
│   │   prompt_tokens=211,
│   │   total_tokens=229,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

In [ ]:
tool_call = response.choices[0].message.tool_calls[0]
tool_id = tool_call.id
arguments = json.loads(tool_call.function.arguments)
query = arguments['query']

result = search_brave(query)
tool_response = {'role':'tool', 'content': result, 'tool_id': tool_id}

We have formatted the next chat message to be a `'tool'` role rather than a `'user'` or `'assistant'` role. Note the `'tool_id'`.

In [ ]:
pprint(tool_response)

{
│   'role': 'tool',
│   'content': "The BEST Pumpkin Pie Recipe - Tastes Better From Scratch\nhttps://tastesbetterfromscratch.com/pumpkin-pie-with-caramel-pecan-topping/\n\nPumpkin Pie Recipe - Preppy Kitchen\nhttps://preppykitchen.com/pumpkin-pie-2/\n\nHomemade Fresh Pumpkin Pie Recipe\nhttps://www.allrecipes.com/recipe/13711/homemade-fresh-pumpkin-pie/\n\nThe Great Pumpkin Pie Recipe - Sally's Baking Addiction\nhttps://sallysbakingaddiction.com/the-great-pumpkin-pie-recipe/\n\nr/thanksgiving on Reddit: What's a good pumpkin pie recipe?\nhttps://www.reddit.com/r/thanksgiving/comments/1fvxp6q/whats_a_good_pumpkin_pie_recipe/\n\n",
│   'tool_id': 'call_dQCZGpf0cm7j3S8cYMrp0jQ2'
}

We first add some additional functionality to our `ChatModel` class so that it can interact with tools. The key section here is that we have to have the following order:

```text
[
    {
        User message,
        Tool completion,
        Tool message (as above),
    }
]
```

As the below is running, we should try to trace through exatly what is happening each step of the way.

In [ ]:
class ChatModel:
    def __init__(self, model: str, system_prompt: str = None):
        self.model = model
        self.client = OpenAI(api_key = api_key)
        self.chat_history: list[dict[str, str]] = []
        
        if system_prompt:
            self.add_message("system", system_prompt)

    def add_message(self, role: str, content: str) -> None:
        self.chat_history.append({
            "role": role,
            "content": content,
        })

    def clear_history(self) -> None:
        self.chat_history = []

    def get_history(self) -> list[dict[str, str]]:
        return self.chat_history
    
    def tool_call(self, tool_call) -> dict:
        tool_id = tool_call.id
        arguments = json.loads(tool_call.function.arguments)

        if tool_call.function.name == 'search_brave':
            print("Searching for information...")
            query = arguments['query']
            result = search_brave(query)

        elif tool_call.function.name == 'scrape_content':
            print("Scraping content...")
            url = arguments['url']
            result = scrape_content(url)
            
        tool_response = {'role':'tool', 'content': result, 'tool_call_id': tool_id}

        return tool_response
    

    def format_tool_response(self, tool_call):
        formatted_dict = {
        "role": "assistant",
        "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": tool_call.type,
                    "function": {
                        "arguments": tool_call.function.arguments,
                        "name": tool_call.function.name
                    }
                }
            ]
        }

        return formatted_dict
    

    def generate(self, message: str, **kwargs) -> str:
        self.add_message("user", message)
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.chat_history,
            **kwargs
        )

        if not response.choices[0].message.content:
            

            tool_completion = response.choices[0].message.tool_calls[0]
            tool_response = self.tool_call(tool_completion)

            messages = self.chat_history + [self.format_tool_response(tool_completion)] + [tool_response]

            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                **kwargs
            )

        assistant_message = response.choices[0].message.content
        self.add_message("assistant", assistant_message)
    
        return response

In [ ]:
model = ChatModel('gpt-4o-mini', system_prompt=system_prompt)

In [ ]:
response = model.generate(
    "What is the best pumpkin pie recipe?",
    max_tokens=512,
    temperature=0.5,
    tools=tools,
)
print(response.choices[0].message.content)

Searching for information...
Here are some highly-rated pumpkin pie recipes:

1. [The BEST Pumpkin Pie Recipe - Tastes Better From Scratch](https://tastesbetterfromscratch.com/pumpkin-pie-with-caramel-pecan-topping/)
2. [Pumpkin Pie Recipe - Preppy Kitchen](https://preppykitchen.com/pumpkin-pie-2/)
3. [Homemade Fresh Pumpkin Pie Recipe - Allrecipes](https://www.allrecipes.com/recipe/13711/homemade-fresh-pumpkin-pie/)
4. [The Great Pumpkin Pie Recipe - Sally's Baking Addiction](https://sallysbakingaddiction.com/the-great-pumpkin-pie-recipe/)
5. [r/thanksgiving on Reddit: What's a good pumpkin pie recipe?](https://www.reddit.com/r/thanksgiving/comments/1fvxp6q/whats_a_good_pumpkin_pie_recipe/)

Would you like more information about any specific recipe?


In [ ]:
# Please note this will likely return an error on KATE!
response = model.generate(
    "Yes, can you summarize the first link for me please?",
    max_tokens=512,
    temperature=0.5,
    tools=tools,
)
print(response.choices[0].message.content)

Scraping content...
The recipe for the BEST Pumpkin Pie from Tastes Better From Scratch is a classic Thanksgiving dessert that is both simple and delicious, featuring an optional caramel pecan topping for added flavor. Here’s a summary of the key points:

### Ingredients:
- **For the Pie:**
  - 1 unbaked 9-inch pie crust (homemade or store-bought)
  - 3/4 cup granulated sugar
  - Spices: 1 tsp cinnamon, 1/2 tsp salt, 1/2 tsp ginger, 1/4 tsp cloves
  - 2 large eggs
  - 15 oz can of pumpkin puree (or fresh pumpkin)
  - 12 oz can of evaporated milk

- **Optional Caramel Pecan Topping:**
  - 1/2 cup light brown sugar
  - 2 tbsp heavy whipping cream
  - 1 tbsp light corn syrup
  - 1 tbsp butter
  - 1/2 cup chopped pecans
  - 1/2 tsp vanilla extract

### Instructions:
1. **Preparation:**
   - Preheat the oven to 425°F (220°C).
   - Beat eggs and pumpkin together in a large bowl. In another bowl, mix sugar and spices, then combine with the pumpkin mixture. Gradually stir in evaporated milk.
 

This is actually a great summary of the ingredients and recipe.

> **Note**: You would probably want to separate your tools from your model, and have a separate class that manages the tools. This would allow you to easily swap out tools, and to manage the tools in a more modular way.

We can also pick a new link, not found by the model, and ask it to scrape that link instead:

In [ ]:
# Please note this will likely return an error on KATE!

new_input = (
    "That's great thanks, but could you also provide me a summary of this link please:\n"
    "https://www.inspiredtaste.net/24962/pumpkin-pie-recipe/"
             )

response = model.generate(
    new_input,
    max_tokens=512,
    temperature=0.5,
    tools=tools,
)
print(response.choices[0].message.content)

Scraping content...
The pumpkin pie recipe from Inspired Taste is celebrated for its simplicity and rich flavor. Here’s a summary of the key points:

### Overview:
- This easy homemade pumpkin pie features a flaky crust and a creamy filling made with heavy cream instead of sweetened condensed milk, resulting in a rich and delicious dessert.

### Key Ingredients:
- **Pie Crust:** Use a homemade or store-bought pie crust. The authors recommend a butter pie crust.
- **Pumpkin:** The recipe works with both canned pumpkin puree (like Libby's) and homemade puree.
- **Eggs:** Three large eggs are used to help the filling set.
- **Sugars:** A mix of granulated and light brown sugar is used for sweetness, which is less than in other recipes.
- **Cream:** Heavy cream is used for a smooth texture.
- **Spices:** The filling includes vanilla, cinnamon, ginger, ground cloves, and salt, allowing the pumpkin flavor to shine.

### Instructions:
1. **Prepare the Crust:** Roll out the pie dough and fit i

In [ ]:
# Please note this will likely return an error on KATE!

response = model.generate(
    "Of these two options, which one would you recommend?",
    max_tokens=512,
    temperature=0.5,
    tools=tools,
)
print(response.choices[0].message.content)

Both pumpkin pie recipes have their unique strengths, so the choice depends on your preferences:

1. **Tastes Better From Scratch:**
   - **Pros:** This recipe offers a traditional approach with a classic flavor profile. The addition of a caramel pecan topping adds a delightful twist if you want something extra special. 
   - **Ideal for:** Those who enjoy a rich, sweet pie with a bit of texture from the pecans and caramel.

2. **Inspired Taste:**
   - **Pros:** This recipe emphasizes simplicity and uses heavy cream for a smooth, creamy filling. It’s straightforward and can be made with either canned or homemade pumpkin. 
   - **Ideal for:** Those who prefer a more traditional pumpkin pie flavor with a silky texture and want a recipe that is easy to follow.

### Recommendation:
- If you’re looking for a classic pumpkin pie experience with an option for a unique topping, go with **Tastes Better From Scratch**.
- If you prefer a simpler recipe that focuses on the creamy texture of the fi